# Financial Transparency System BGUE Budgets (2016)

In [ ]:
import json

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

## Load Data and Restrict to UK

In [ ]:
all_fts = pd.read_excel('input/export_2016_en.xlsx')
all_fts.shape

Note from looking at the data: This dataset is grouped up with merged cells. Whilst some have an amount per row, others only have a total amount. 

In [ ]:
all_fts.columns

In [ ]:
# Rename columns to more usable format.
all_fts.rename({
    'Country / Territory': 'country', 
    'Reference of the Legal Commitment (LC)': 'reference_of_lc', 
    'Commitment position key': 'commitment_position_key',
    'Year': 'year',
    'Name of beneficiary': 'beneficiary',
    'Coordinator': 'coordinator',
    'VAT Number of beneficiary': 'beneficiary_vat', 
    'Postal code': 'raw_postcode',
    'Amount': 'amount',
    'Source of (estimated) detailed amount': 'source_of_amount', 
    'Geographical Zone': 'geographical_zone',
    'Expense Type': 'expense_type',
    'Total amount': 'total_amount', 
    'Subject of grant or contract': 'subject_of_grant_or_contract',
    'Responsible Department': 'responsible_department', 
    'Budget line name and number': 'budget_line_name_and_number',
    'Action Type': 'action_type',
    'Funding Type': 'funding_type',
    'LE Acct Group Code': 'le_acct_group_code', 
    'LE Acct Group Desc': 'le_acct_group_desc'
}, axis=1, inplace=True)

In [ ]:
# The `total_amount` comes up zero rather than NA when the cell is unmerged.
all_fts.total_amount.isna().sum()

In [ ]:
# Is it safe to replace zeros with NaNs and then fill?
cps_total_amounts = all_fts.groupby('commitment_position_key', sort=False).aggregate({'total_amount': sum})
nonzero_total_amounts = all_fts.total_amount[all_fts.total_amount > 0]
[
    cps_total_amounts.total_amount.shape[0],
    nonzero_total_amounts.values.shape[0],    
    (cps_total_amounts.total_amount.values == nonzero_total_amounts.values).sum()
]

In [ ]:
all_fts.loc[all_fts.total_amount == 0, 'total_amount'] = float('nan')

In [ ]:
# Forward fill amounts, as there are merged cells in the original 2016 database
# which are being filled with NaNs.
all_fts.commitment_position_key = all_fts.commitment_position_key.fillna(method='ffill')
all_fts.year = all_fts.year.fillna(method='ffill')
all_fts.expense_type = all_fts.expense_type.fillna(method='ffill')
all_fts.total_amount = all_fts.total_amount.fillna(method='ffill')
all_fts.subject_of_grant_or_contract = all_fts.subject_of_grant_or_contract.fillna(method='ffill')
all_fts.responsible_department = all_fts.responsible_department.fillna(method='ffill')
all_fts.budget_line_name_and_number = all_fts.budget_line_name_and_number.fillna(method='ffill')
all_fts.action_type = all_fts.action_type.fillna(method='ffill')
all_fts.funding_type = all_fts.funding_type.fillna(method='ffill')

In [ ]:
all_fts.country.sort_values().unique()

In [ ]:
fts = all_fts[all_fts.country == 'United Kingdom'].copy()
fts.shape

In [ ]:
fts.head()

## Columns

### `reference_of_lc`

Always missing.

In [ ]:
fts.reference_of_lc.unique()

### `year`

All 2016.

In [ ]:
fts.year.isna().sum()

In [ ]:
fts.year.unique()

### `beneficiary`

Many seem to have asterisks; we may want to get rid of content after the asterisk.

In [ ]:
fts.beneficiary.isna().sum()

In [ ]:
(fts.beneficiary != fts.beneficiary.str.strip()).sum()

In [ ]:
(fts.beneficiary != fts.beneficiary.str.upper()).sum()

In [ ]:
[fts.shape, len(fts.beneficiary.unique())]

In [ ]:
fts.beneficiary.str.contains('*', regex=False).sum()

In [ ]:
fts[fts.beneficiary.str.contains('*', regex=False)].beneficiary.unique()[0:20]

### `Type`

Mostly missing. Not that interesting.

In [ ]:
fts.Type.isna().sum()

In [ ]:
fts.Type.unique()

### `Coordinator`

In [ ]:
fts.coordinator.isna().sum()

In [ ]:
fts.coordinator.unique()

### `amount`

Probably what we are most interested in. Missing for some, where only a total amount is known but not the per-beneficiary amount.

In [ ]:
[fts.shape, fts.amount.isna().sum()]

In [ ]:
fts.amount.describe()

In [ ]:
(fts.amount == 0).sum()

In [ ]:
fts[fts.amount == 0].head()

### `source_of_amount`

Not present for any UK data.

In [ ]:
fts.source_of_amount.isna().sum()

### `NUTS2`

Where the `raw_postcode` is censored, we have a NUTS2. This is not very easy for us to use, but maybe in future we could try to use it.

In [ ]:
(~fts.NUTS2.isna()).sum()

In [ ]:
fts[~fts.NUTS2.isna()].head()

### `geographical_zone`

Present for some international aide projects. May be worth including.

In [ ]:
fts.geographical_zone.isna().sum()

In [ ]:
fts.geographical_zone.sort_values().unique()

In [ ]:
fts[~fts.geographical_zone.isna()].head()

### `expense_type`

In [ ]:
fts.expense_type.isna().sum()

In [ ]:
fts.expense_type.unique()

### `total_amount`

Always present, but often zero. Not sure

In [ ]:
fts.total_amount.isna().sum()

In [ ]:
fts.total_amount.describe()

In [ ]:
(fts.amount > fts.total_amount).sum()

In [ ]:
(fts.amount / fts.total_amount).describe()

### `subject_of_grant_or_contract`

In [ ]:
fts.subject_of_grant_or_contract.isna().sum()

In [ ]:
[fts.shape, fts.subject_of_grant_or_contract.unique().shape]

In [ ]:
fts[fts.subject_of_grant_or_contract.duplicated(keep=False)].sort_values('subject_of_grant_or_contract').head()

### `responsible_department`

In [ ]:
fts.responsible_department.isna().sum()

In [ ]:
fts.responsible_department.unique().shape

In [ ]:
fts.groupby('responsible_department').size().\
    reset_index(name='size').sort_values('size', ascending=False).head(10)

### `budget_line_name_and_number`

In [ ]:
fts.budget_line_name_and_number.isna().sum()

In [ ]:
fts.budget_line_name_and_number.unique().shape

In [ ]:
fts.groupby('budget_line_name_and_number').size().\
    reset_index(name='size').sort_values('size', ascending=False).head(10)

In [ ]:
fts.groupby(['responsible_department', 'budget_line_name_and_number']).size().\
    reset_index(name='size').sort_values('size', ascending=False).head(10)

### `action_type`

In [ ]:
fts.action_type.isna().sum()

In [ ]:
fts.action_type.unique().shape

In [ ]:
fts.groupby(['action_type']).size().\
    reset_index(name='size').sort_values('size', ascending=False).head(20)

### `funding_type`

In [ ]:
fts.funding_type.isna().sum()

In [ ]:
fts.funding_type.unique()

### Overlaps with other datasets

### `creative europe`

Overlap here between the two datasets, so we will remove

In [ ]:
fts_creative =  fts[fts.action_type.str.contains('Creative')]
fts_creative.shape

In [ ]:
fts_creative.action_type.unique()

In [ ]:
fts_creative.head()

In [ ]:
fts =  fts[~fts.action_type.str.contains('Creative')]
fts.shape

### `horizon 2020`

In [ ]:
fts_h2020 =  fts[fts.action_type.str.contains('Horizon 2020')]
fts_h2020.shape

In [ ]:
fts_h2020.action_type.unique()

In [ ]:
fts_h2020.head()

In [ ]:
fts =  fts[~fts.action_type.str.contains('Horizon 2020')]
fts.shape

### `FP7`

Research: Framework programme n°7 - again overlap with existing data, so removing these	

In [ ]:
fts_fp7 =  fts[fts.action_type.str.contains('n°7')]
fts_fp7.shape

In [ ]:
fts_fp7.action_type.unique()

In [ ]:
fts_fp7.head()

In [ ]:
fts =  fts[~fts.action_type.str.contains('n°7')]
fts.shape

### `Erasmus +`


In [ ]:
fts_erasmus =  fts[fts.action_type.str.contains('Erasmus')]
fts_erasmus.shape

In [ ]:
fts_erasmus.action_type.unique()

In [ ]:
fts = fts[~fts.action_type.str.contains('Erasmus')]
fts.shape

### `Research and Innovation`

From what I can see there is no overlap here, so not removing

In [ ]:
fts_randi =  fts[fts.action_type.str.contains('08 Research and innovation')]
fts_randi.shape

In [ ]:
fts_randi[fts_randi.raw_postcode != '*****'].head()

## Filter projects with funding < €1000

In [ ]:
fts[fts.total_amount < 1000].shape

In [ ]:
fts = fts[fts.total_amount >= 1000]
fts.shape

### Drop uninteresting columns

In [ ]:
clean_fts = fts.drop([
    'reference_of_lc', 'commitment_position_key', 'year',
    'Type', 'beneficiary_vat', 'Address', 'City', 'country',
    'commitment_position_key', 'source_of_amount',
    'le_acct_group_code', 'le_acct_group_desc'
], axis=1)
clean_fts.head()

### `postcode`

In [ ]:
ukpostcodes = pd.read_csv('../postcodes/input/ukpostcodes.csv.gz')
ukpostcodes.shape

In [ ]:
clean_fts.raw_postcode.isin(ukpostcodes.postcode).sum()

In [ ]:
clean_fts['postcode'] = clean_fts.raw_postcode.\
    str.upper().\
    str.strip().\
    str.replace(r'[^A-Z0-9]', '').\
    str.replace(r'^(\S+)([0-9][A-Z]{2})$', r'\1 \2')

In [ ]:
clean_fts.postcode.isin(ukpostcodes.postcode).sum()

In [ ]:
clean_fts.raw_postcode[~clean_fts.postcode.isin(ukpostcodes.postcode)].unique()

Some digit substitutions and some deprecated, e.g. `CB4 9ZR`.

## Convert to GBP

All we have is a year, so just use the average annual exchange rate.

In [ ]:
eur_gbp = pd.read_pickle('../exchange_rates/output/exchange_rates.pkl.gz')
eur_gbp.tail()

In [ ]:
def find_average_eur_gbp_rate():
    # create timeseries from start to end
    days = pd.date_range('2016-01-01', '2017-01-01', closed='left')
    daily = pd.DataFrame({
        'month_start': days,
        'weight': 1.0 / days.shape[0]
    })
    monthly = daily.resample('MS', on='month_start').sum()
    monthly = pd.merge(monthly, eur_gbp, on='month_start', validate='1:1')
    return (monthly.weight * monthly.rate).sum()
clean_fts['eur_gbp'] = find_average_eur_gbp_rate()
clean_fts.eur_gbp.head()

## Output

In [ ]:
output_fts = clean_fts[
    clean_fts.postcode.isin(ukpostcodes.postcode) &
    (clean_fts.amount > 0)
].copy()
output_fts.drop(['raw_postcode', 'NUTS2'], axis=1, inplace=True)
output_fts.rename(columns={'total_amount': 'total_amount_eur'}, inplace=True)
output_fts.shape

In [ ]:
output_fts.head()

In [ ]:
output_fts['my_eu_id'] = 'fts_2016_' + output_fts.index.map(str)
output_fts.my_eu_id.head()

In [ ]:
output_fts.to_pickle('output/fts_2016.pkl.gz')